In [ ]:
import json
import pandas as pd
import os
from tqdm import tqdm
from datetime import datetime

# 1. 경로 설정 
BASE_PATH = "/home/ec2-user/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/264"
INPUT_FILE = os.path.join(BASE_PATH, 'arxiv-metadata-oai-snapshot.json')
OUTPUT_FILE = 'arxiv_cs_recent_filtered.csv'

# 필터링 조건
TARGET_CATEGORIES = {'cs.CV', 'cs.CL', 'cs.LG', 'cs.AI', 'cs.RO'} 
START_YEAR = 2020

def parse_arxiv_data():
    data = []
    print(f"📂 읽어올 파일 경로: {INPUT_FILE}")
    print("🚀 데이터 필터링 시작 (CS 분야 & 2020년 이후)...")
    
    # 파일 존재 여부 확인
    if not os.path.exists(INPUT_FILE):
        print(f"❌ 오류: 해당 경로에 파일이 없습니다. 경로를 다시 확인해주세요: {INPUT_FILE}")
        return

    with open(INPUT_FILE, 'r') as f:
        for line in tqdm(f, desc="Processing"):
            try:
                doc = json.loads(line)
                
                # 1. 카테고리 필터링 (속도 최적화를 위해 먼저 체크)
                if not 'cs.' in doc['categories']: 
                    continue
                
                cats = set(doc['categories'].split())
                if not cats.intersection(TARGET_CATEGORIES):
                    continue
                
                # 2. 날짜 필터링
                update_date = datetime.strptime(doc['update_date'], "%Y-%m-%d")
                if update_date.year < START_YEAR:
                    continue
                
                # 3. 데이터 확보
                data.append({
                    'id': doc['id'],
                    'title': doc['title'].replace('\n', ' ').strip(),
                    'abstract': doc['abstract'].replace('\n', ' ').strip(),
                    'categories': doc['categories'],
                    'year': update_date.year,
                    'date': doc['update_date']
                })
            except Exception as e:
                continue

    # CSV 저장
    if data:
        df = pd.DataFrame(data)
        print(f"\n📊 추출된 논문 수: {len(df)}건 ({START_YEAR}~현재)")
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"✅ 저장 완료: {OUTPUT_FILE} (이 파일은 현재 폴더에 생성됨)")
    else:
        print("⚠️ 조건에 맞는 데이터가 없습니다. 필터링 조건을 확인해주세요.")

# 실행
if __name__ == "__main__":
    parse_arxiv_data()

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

DATA_FILE = 'arxiv_cs_recent_filtered.csv'
EMBEDDING_FILE = 'sbert_embeddings_sampled_all-MiniLM-L6-v2.npy' 

print("📂 데이터 로딩 중...")
df = pd.read_csv(DATA_FILE)
embeddings = np.load(EMBEDDING_FILE)
print("✅ 데이터 로드 완료")

# 모델 로드 (검색용)
model = SentenceTransformer('all-MiniLM-L6-v2')
if torch.cuda.is_available():
    model = model.to('cuda')

def search_similar_papers(query_text, top_k=5):
    # Query 인코딩
    query_vec = model.encode([query_text], convert_to_numpy=True, normalize_embeddings=True)
    
    # 유사도 계산 (Dot Product)
    scores = np.dot(query_vec, embeddings.T).flatten()
    
    # 상위 K개 추출
    top_indices = np.argsort(scores)[::-1][:top_k]
    
    print(f"\n🔍 Query: {query_text[:100]}...")
    print("-" * 80)
    
    for rank, idx in enumerate(top_indices):
        paper = df.iloc[idx]
        score = scores[idx]
        # 결과 출력
        print(f"[{rank+1}] 유사도: {score:.4f} | {paper['year']} | {paper['categories']}")
        print(f"Title: {paper['title']}")
        print(f"Link: https://arxiv.org/abs/{paper['id']}")
        print("-" * 80)

my_abstract = """
We propose a novel method for split federated learning to address data heterogeneity.
Our approach uses gradient shuffling to improve convergence speed on non-IID data.
"""
search_similar_papers(my_abstract)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


📂 데이터 로딩 중...


/tmp/ipykernel_19085/2255113780.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_FILE)


✅ 데이터 로드 완료


NameError: name 'torch' is not defined

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

DATA_FILE = 'arxiv_cs_recent_filtered.csv' 
# DATA_FILE = 'arxiv_cs_recent_filtered.csv' 

def extract_distinctive_keywords():
    print(f"📂 데이터 로드 중... ({DATA_FILE})")
    try:
        df = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return

    # 불용어(의미 없는 단어) 추가 제거
    custom_stopwords = list(ENGLISH_STOP_WORDS) + [
        'propose', 'method', 'paper', 'based', 'approach', 'results', 'model', 
        'performance', 'state', 'art', 'using', 'novel', 'learning', 'network',
        'algorithm', 'models', 'proposed', 'data', 'experiments', 'show', 'task',
        'neural', 'networks', 'deep', 'analysis', 'applications', 'study', 'use'
    ]
    
    years = sorted(df['year'].unique())
    keywords_by_year = {} # 연도별 키워드 저장소
    
    print("📊 연도별 키워드 분석 및 공통 단어 필터링 중...")
    
    # 1. 각 연도별 Top 20 추출하여 저장
    for year in years:
        year_docs = df[df['year'] == year]['title']
        combined_text = " ".join(year_docs.fillna(""))
        
        # 빈도 분석
        vectorizer = CountVectorizer(stop_words=custom_stopwords, max_features=1000, ngram_range=(1, 2))
        try:
            X = vectorizer.fit_transform([combined_text])
            feature_names = vectorizer.get_feature_names_out()
            
            # 빈도순 정렬 및 Top 20 추출
            row = X[0].toarray().flatten()
            top_indices = row.argsort()[::-1][:50] 
            top_keywords = [feature_names[idx] for idx in top_indices]
            
            keywords_by_year[year] = top_keywords
        except ValueError:
            # 데이터가 너무 적어서 어휘가 생성 안 된 경우
            keywords_by_year[year] = []

    # 2. 모든 연도에 공통적으로 등장한 키워드(교집합) 찾기
    if keywords_by_year:
        # 첫 번째 해의 키워드를 기준으로 시작해서 계속 교집합(intersection)을 구함
        common_keywords = set(keywords_by_year[years[0]])
        for year in years[1:]:
            common_keywords = common_keywords.intersection(set(keywords_by_year[year]))
            
        print("\n" + "="*80)
        print(f"🚫 제외된 공통 키워드 (모든 연도 등장): {', '.join(common_keywords)}")
        print("="*80 + "\n")

        # 3. 공통 키워드 제거 후 출력
        print(f"{'Year':<6} | {'Distinctive Trending Keywords (Unique per Year)':<60}")
        print("-" * 80)
        
        for year in years:
            # 원래 리스트 순서(빈도순)를 유지하면서 공통 키워드만 제거
            unique_list = [k for k in keywords_by_year[year] if k not in common_keywords]
            
            # 너무 많이 빠져서 리스트가 짧아질 수 있으므로 상위 10~15개 정도만 문자열로 변환
            unique_str = ', '.join(unique_list[:15]) 
            print(f"{year:<6} | {unique_str}")
            
        print("="*80)

if __name__ == "__main__":
    extract_distinctive_keywords()